In [1]:
import ipywidgets as widgets
import pandas as pd
import json
import torch

from datasets import load_dataset

with open("mergedJson.jsonl", "r") as jsonl_file:
    lines = jsonl_file.readlines()

data_list = []
for line in lines:
    data = json.loads(line)
    data_list.append(data)

dataframe = pd.DataFrame(data_list, columns=["GENERATED_DESCRIPTION", "GENERATED_DATA"])

dataframe

,GENERATED_DESCRIPTION,GENERATED_DATA
0,"{'large_cube_location': '<Vector (0.0000, 0.00...",{'COLLADA': {'@xmlns': 'http://www.collada.org...
1,"{'large_cube_location': '<Vector (0.0000, 0.00...",{'COLLADA': {'@xmlns': 'http://www.collada.org...
2,"{'large_cube_location': '<Vector (0.0000, 0.00...",{'COLLADA': {'@xmlns': 'http://www.collada.org...
3,"{'large_cube_location': '<Vector (0.0000, 0.00...",{'COLLADA': {'@xmlns': 'http://www.collada.org...
4,"{'large_cube_location': '<Vector (0.0000, 0.00...",{'COLLADA': {'@xmlns': 'http://www.collada.org...
...,...,...
995,"{'large_cube_location': '<Vector (0.0000, 0.00...",{'COLLADA': {'@xmlns': 'http://www.collada.org...
996,"{'large_cube_location': '<Vector (0.0000, 0.00...",{'COLLADA': {'@xmlns': 'http://www.collada.org...
997,"{'large_cube_location': '<Vector (0.0000, 0.00...",{'COLLADA': {'@xmlns': 'http://www.collada.org...
998,"{'large_cube_location': '<Vector (0.0000, 0.00...",{'COLLADA': {'@xmlns': 'http://www.collada.org...


In [2]:
# From https://huggingface.co/docs/transformers/model_doc/t5#training
# and also
# https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/T5/Fine_tune_CodeT5_for_generating_docstrings_from_Ruby_code.ipynb

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("Salesforce/codet5-small")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")

max_source_length = 3072
max_target_length = 5176

# task_prefix = "json2dae: "


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def preprocess_data(inputData):
    input_sequences = inputData["GENERATED_DESCRIPTION"]
    input_sequences = input_sequences.astype(str)
    
    output_sequences = inputData["GENERATED_DATA"]
    output_sequencs = output_sequences.astype(str)
    
    model_inputs = tokenizer(
        input_sequences, 
        max_length = max_source_length, 
        padding = "max_length",
        truncation = True
    )
    
    labels = tokenizer(
        output_sequences,
        max_length = max_target_length,
        padding = "max_length",
        truncation = True
    )
    
    # Replace padding tokens with -100 so they are
    # ignored by CrossEntropyLoss thing
    labels_with_ignore_index = []
    for sample in labels:
        sample = [label if label != 0 else -100 for label in sample[
                  labels_with_ignore_index.append[sample]
    
    model_inputs["labels"] = labels_with_ignore_index
    return model_inputs

In [ ]:
mapped_dataset = dataframe.map(preprocess_data, batched=True)
mapped_dataset